In [2]:
import pandas as pd
import os
from datetime import datetime, timedelta

# ==========================================
# 설정 (Settings)
# ==========================================
symbol = "BTCUSDT"
date_str = "2025-03-01"
top_n = 50
window = 60 # 검증할 윈도우 크기 (60분)

# 파일 경로
x_path = f"data/x/{date_str}_x_top{top_n}.h5"
raw_dir = "data/1m_raw_data"

# ==========================================
# 1. 데이터 로드 및 병합
# ==========================================
print(f"[INFO] {symbol} 데이터 추출 중...")

# (1) X 데이터 (Python이 계산한 Feature) 로드
if not os.path.exists(x_path):
 raise FileNotFoundError(f"X 파일 없음: {x_path}")

df_x = pd.read_hdf(x_path, key="x")
df_x = df_x[df_x["symbol"] == symbol].copy()

# Feature 컬럼만 선택 (중립화된 _neut 제외, 원본 Feature 값만 확인)
feat_cols = [c for c in df_x.columns if c.startswith(f"x_{window}m_") and not c.endswith("_neut")]
# 비교를 위해 필요한 시간, 심볼 정보
df_x_subset = df_x[["start_time_ms", "start_time_utc"] + feat_cols].copy()

# (2) Raw 데이터 (OHLCV 원본) 로드
# 윈도우 계산을 위해 전날 데이터도 필요
target_date = datetime.strptime(date_str, "%Y-%m-%d").date()
prev_date_str = (target_date - timedelta(days=1)).strftime("%Y-%m-%d")
raw_paths = [os.path.join(raw_dir, f"{d}.h5") for d in [prev_date_str, date_str]]

raw_list = []
for p in raw_paths:
 if os.path.exists(p):
 raw_list.append(pd.read_hdf(p))

if not raw_list:
 raise ValueError("원본 데이터 파일이 없습니다.")

df_raw = pd.concat(raw_list, ignore_index=True)
df_raw = df_raw[df_raw["symbol"] == symbol].copy()
df_raw = df_raw.sort_values("start_time_ms")

# (3) 두 데이터 병합 (Time 기준)
# Raw 데이터에 Feature 데이터를 붙임
merged = pd.merge(df_raw, df_x_subset, on=["start_time_ms", "start_time_utc"], how="left")

# 엑셀에서 보기 편하게 타겟 날짜 데이터만 남기되, 
# 엑셀 수식 검증을 위해 앞쪽 데이터(전날 23시~)도 일부 포함하여 저장
cutoff_time = df_x_subset["start_time_ms"].min() - (window * 60 * 1000 * 2) # 넉넉하게 윈도우 2배 전부터
merged_save = merged[merged["start_time_ms"] >= cutoff_time].copy()

# ==========================================
# 2. 엑셀로 내보내기
# ==========================================
output_file = f"{symbol}_verification_window{window}.xlsx"
merged_save.to_excel(output_file, index=False)

print(f"[DONE] 파일 생성 완료: {output_file}")

 # 3월 1일의 BTCUSDT 의 60min window에 대해 

 # feature가 올바르게 계산되었는 지 확인하기 위해 excel 수식을 이용해 검증하였음

### Feature Checker for CloseDiffRatio, RatioSkewZ, OI_Z 

In [ ]:
import pandas as pd
import numpy as np
import os
import json

HDF5_PATH = 'data/x/2025-03-04_x_top30.h5'
target_timestamp = 1741132620000
df_x = pd.read_hdf(HDF5_PATH, key='x')


raw_files = ['data/1m_raw_data/2025-03-03.h5', 'data/1m_raw_data/2025-03-04.h5']
raw_dfs = []
for rf in raw_files:
    if os.path.exists(rf):
        try:
            raw_dfs.append(pd.read_hdf(rf))
        except:
            import h5py
            with h5py.File(rf, 'r') as f:
                keys = list(f.keys())
            raw_dfs.append(pd.read_hdf(rf, key=keys[0]))

In [2]:
btc = df_x[df_x['symbol']=='BTCUSDT']
btc[btc['start_time_ms']==1741132620000]

window = 15

In [3]:
raw = raw_dfs[1]
raw[raw['symbol']=='BTCUSDT']
btc_raw = raw[raw['symbol']=='BTCUSDT'].sort_values('start_time_ms').reset_index(drop=True)
# get index of target timestamp
idx = btc_raw.index[btc_raw['start_time_ms']==1741132620000]

In [4]:
btc_raw.iloc[idx[0]-(window-1):idx[0]+1]

,symbol,start_time_ms,start_time_utc,open,high,low,close,volume_base,quote_volume,num_trades,...,taker_buy_quote,pm_open,pm_high,pm_low,pm_close,mt_oi,mt_top_ls_ratio_cnt,mt_top_ls_ratio,mt_ls_ratio_cnt,mt_ls_vol_ratio
1423,BTCUSDT,1741131780000,2025-03-04 23:43:00,87301.0,87307.5,87266.9,87307.3,45.199,3.945187e+06,1092,...,9.195444e+05,-0.000397,-0.000164,-0.000563,-0.000340,5.916980e+09,1.990885,1.984824,1.788079,0.661138
1424,BTCUSDT,1741131840000,2025-03-04 23:44:00,87307.3,87380.0,87285.8,87373.5,71.212,6.217652e+06,1542,...,3.192032e+06,-0.000342,-0.000195,-0.000349,-0.000215,5.916980e+09,1.990885,1.984824,1.788079,0.661138
1425,BTCUSDT,1741131900000,2025-03-04 23:45:00,87373.4,87373.5,87300.6,87348.5,66.271,5.787315e+06,1497,...,2.116897e+06,-0.000400,-0.000066,-0.000463,-0.000338,5.915656e+09,1.988307,1.984456,1.783261,0.563536
1426,BTCUSDT,1741131960000,2025-03-04 23:46:00,87348.5,87348.5,87288.5,87290.6,46.623,4.070814e+06,1141,...,2.162037e+06,-0.000389,-0.000230,-0.000709,-0.000313,5.915656e+09,1.988307,1.984456,1.783261,0.563536
1427,BTCUSDT,1741132020000,2025-03-04 23:47:00,87290.6,87310.0,87260.0,87274.5,51.471,4.493036e+06,1328,...,2.551441e+06,-0.000630,-0.000209,-0.000630,-0.000498,5.915656e+09,1.988307,1.984456,1.783261,0.563536
1428,BTCUSDT,1741132080000,2025-03-04 23:48:00,87274.5,87307.4,87269.6,87271.7,37.209,3.247564e+06,1053,...,1.711242e+06,-0.000421,-0.000163,-0.000505,-0.000391,5.915656e+09,1.988307,1.984456,1.783261,0.563536
1429,BTCUSDT,1741132140000,2025-03-04 23:49:00,87271.8,87350.0,87271.7,87332.4,31.357,2.738213e+06,1285,...,1.364968e+06,-0.000441,-0.000225,-0.000441,-0.000348,5.915656e+09,1.988307,1.984456,1.783261,0.563536
1430,BTCUSDT,1741132200000,2025-03-04 23:50:00,87332.4,87360.0,87300.2,87359.9,62.134,5.426405e+06,1574,...,1.917072e+06,-0.000459,-0.000148,-0.000886,-0.000148,5.914453e+09,1.981589,1.982832,1.775578,0.949864
1431,BTCUSDT,1741132260000,2025-03-04 23:51:00,87360.0,87360.0,87280.0,87280.1,29.198,2.549465e+06,1316,...,1.312992e+06,-0.000333,-0.000272,-0.000650,-0.000436,5.914453e+09,1.981589,1.982832,1.775578,0.949864
1432,BTCUSDT,1741132320000,2025-03-04 23:52:00,87280.0,87299.9,87270.5,87292.9,45.099,3.936322e+06,1125,...,2.411770e+06,-0.000460,-0.000292,-0.000484,-0.000461,5.914453e+09,1.981589,1.982832,1.775578,0.949864


In [5]:
# --- Clear Manual Verification for BTCUSDT (15m Window) ---
window = 15

# Retrieve variables from previous cells
target_idx = idx[0]
target_window = window
target_ts = 1741132620000

# 1. Define the Window Slice
# Window is [t - window + 1, t]
start_pos = target_idx - target_window + 1
end_pos = target_idx

window_slice = btc_raw.iloc[start_pos : end_pos + 1]

print(f"Verifying {target_window}m_Close_Diff_Rate for BTCUSDT at {target_ts}")
print(f"Window Indices: {start_pos} to {end_pos} (Size: {len(window_slice)})")
print(f"Time Range: {window_slice['start_time_ms'].min()} -> {window_slice['start_time_ms'].max()}")

# 2. Get Values
c_first = window_slice['close'].iloc[0]
c_last = window_slice['close'].iloc[-1]

print(f"\n[Raw Data]")
print(f"  First Close (t-{target_window-1}): {c_first}")
print(f"  Last Close  (t):    {c_last}")

# 3. Calculate
manual_calc = (c_last / c_first) - 1.0
print(f"\n[Calculation]")
print(f"  ({c_last} / {c_first}) - 1.0 = {manual_calc:.10f}")

# 4. Compare with Model
# Check for column name (sometimes prefixed with x_)
col_name = f"{target_window}m_Close_Diff_Rate"
if col_name not in btc.columns:
     col_name = f"x_{target_window}m_Close_Diff_Rate"

if col_name in btc.columns:
    model_val = btc[btc['start_time_ms'] == target_ts][col_name].values[0]
    
    print(f"\n[Comparison]")
    print(f"  Manual: {manual_calc:.10f}")
    print(f"  Model:  {model_val:.10f}")
    print(f"  Diff:   {abs(manual_calc - model_val):.10e}")
else:
    print(f"\nColumn {col_name} not found in model data.")

Verifying 15m_Close_Diff_Rate for BTCUSDT at 1741132620000
Window Indices: 1423 to 1437 (Size: 15)
Time Range: 1741131780000 -> 1741132620000

[Raw Data]
  First Close (t-14): 87307.3
  Last Close  (t):    87252.5

[Calculation]
  (87252.5 / 87307.3) - 1.0 = -0.0006276680

[Comparison]
  Manual: -0.0006276680
  Model:  -0.0006276680
  Diff:   0.0000000000e+00


In [6]:
# --- Verify 15m_RatioSkew_Z ---
window = 15
target_ts = 1741132620000

# Ensure we have the target index
if len(btc_raw.index[btc_raw['start_time_ms'] == target_ts]) == 0:
    print(f"Target timestamp {target_ts} not found in btc_raw")
else:
    target_idx = btc_raw.index[btc_raw['start_time_ms'] == target_ts][0]

    # We need ratio_skew values for the last 'window' steps to calculate Z-score at target_idx
    # ratio_skew at index i uses data from [i - window + 1 : i]
    # So we need to iterate i from (target_idx - window + 1) to target_idx

    ratio_skew_values = []
    indices_for_z = range(target_idx - window + 1, target_idx + 1)

    print(f"Calculating RatioSkew for {len(indices_for_z)} steps to compute Z-score...")

    for i in indices_for_z:
        # Slice for RatioSkew calculation at index i
        # Data needed: [i - window + 1 : i]
        start_p = i - window + 1
        end_p = i
        
        # Ensure we have data (handle negative indices if necessary, though unlikely here)
        if start_p < 0:
            # If we need data from previous file, this simple check fails. 
            # But assuming we are late in the file:
            ratio_skew_values.append(0.0) 
            continue
            
        slice_df = btc_raw.iloc[start_p : end_p + 1]
        
        # Calculate RatioSkew
        # ratio_skew = mean(mt_top_ls_ratio) - mean(mt_ls_ratio_cnt)
        if 'mt_top_ls_ratio' in slice_df.columns and 'mt_ls_ratio_cnt' in slice_df.columns:
            top_ls_mean = slice_df['mt_top_ls_ratio'].mean()
            ls_mean = slice_df['mt_ls_ratio_cnt'].mean()
            r_skew = top_ls_mean - ls_mean
            ratio_skew_values.append(r_skew)
        else:
            ratio_skew_values.append(0.0)

    # Now calculate Z-score of the last value in ratio_skew_values
    # using the window of ratio_skew_values
    rs_series = pd.Series(ratio_skew_values)

    # Manual Z-score
    # The Z-score at the last step uses the mean/std of the last 'window' steps of ratio_skew
    # Since we collected exactly 'window' steps, we use the whole series
    rs_mean = rs_series.mean()
    rs_std = rs_series.std(ddof=0) + 1e-9
    rs_last = rs_series.iloc[-1]

    manual_z = (rs_last - rs_mean) / rs_std

    print(f"\n[RatioSkew Data (Last 5)]")
    print(rs_series.tail().values)
    print(f"\n[Z-Score Calculation]")
    print(f"  Mean: {rs_mean:.10f}")
    print(f"  Std:  {rs_std:.10f}")
    print(f"  Last: {rs_last:.10f}")
    print(f"  Z:    ({rs_last} - {rs_mean}) / {rs_std} = {manual_z:.10f}")

    # Compare
    col_name = f"{window}m_RatioSkew_Z"
    # Check if column exists (sometimes prefixed)
    if col_name not in btc.columns:
         col_name = f"x_{window}m_RatioSkew_Z" # Just in case, though x_generator output usually doesn't have x_ prefix in columns inside the file, but let's be safe

    if col_name in btc.columns:
        model_val = btc[btc['start_time_ms'] == target_ts][col_name].values[0]
        print(f"\n[Comparison]")
        print(f"  Manual: {manual_z:.10f}")
        print(f"  Model:  {model_val:.10f}")
        print(f"  Diff:   {abs(manual_z - model_val):.10e}")
    else:
        print(f"Column {col_name} not found in model data.")

Calculating RatioSkew for 15 steps to compute Z-score...

[RatioSkew Data (Last 5)]
[0.20058231 0.20173131 0.2024918  0.20325228 0.20401277]

[Z-Score Calculation]
  Mean: 0.1973955108
  Std:  0.0042355087
  Last: 0.2040127673
  Z:    (0.20401276733333296 - 0.19739551075555553) / 0.004235508691580015 = 1.5623286504

[Comparison]
  Manual: 1.5623286504
  Model:  1.5623286504
  Diff:   2.5083490840e-11


In [7]:
# --- Verify 15m_OI_Z ---
window = 15
target_ts = 1741132620000

# Ensure we have the target index
if len(btc_raw.index[btc_raw['start_time_ms'] == target_ts]) == 0:
    print(f"Target timestamp {target_ts} not found in btc_raw")
else:
    target_idx = btc_raw.index[btc_raw['start_time_ms'] == target_ts][0]
    
    # Window slice for OI
    # Data needed: [target_idx - window + 1 : target_idx]
    start_pos = target_idx - window + 1
    end_pos = target_idx
    
    window_slice = btc_raw.iloc[start_pos : end_pos + 1]
    
    if 'mt_oi' in window_slice.columns:
        oi_series = window_slice['mt_oi']
        
        # Calculate components
        oi_last = oi_series.iloc[-1]
        oi_mean = oi_series.mean()
        oi_std = oi_series.std(ddof=0) + 1e-9
        
        # Formula from x_generator.py:
        # oi_z = (oi_last / oi_mean - 1.0) / (oi_std / oi_mean)
        
        
        manual_oi_z = (oi_last / oi_mean - 1.0) / (oi_std / oi_mean)
        
        print(f"Verifying {window}m_OI_Z for BTCUSDT at {target_ts}")
        print(f"\n[OI Data]")
        print(f"  Last: {oi_last}")
        print(f"  Mean: {oi_mean}")
        print(f"  Std:  {oi_std}")
        
        print(f"\n[Calculation]")
        print(f"  ({oi_last} / {oi_mean} - 1.0) / ({oi_std} / {oi_mean}) = {manual_oi_z:.10f}")
        
        # Compare
        col_name = f"{window}m_OI_Z"
        if col_name not in btc.columns:
             col_name = f"x_{window}m_OI_Z"

        if col_name in btc.columns:
            model_val = btc[btc['start_time_ms'] == target_ts][col_name].values[0]
            print(f"\n[Comparison]")
            print(f"  Manual: {manual_oi_z:.10f}")
            print(f"  Model:  {model_val:.10f}")
            print(f"  Diff:   {abs(manual_oi_z - model_val):.10e}")
        else:
            print(f"Column {col_name} not found in model data.")
            
    else:
        print("Column 'mt_oi' not found in raw data.")

Verifying 15m_OI_Z for BTCUSDT at 1741132620000

[OI Data]
  Last: 5911336510.674255
  Mean: 5914567802.818707
  Std:  1806426.2212936282

[Calculation]
  (5911336510.674255 / 5914567802.818707 - 1.0) / (1806426.2212936282 / 5914567802.818707) = -1.7887761517

[Comparison]
  Manual: -1.7887761517
  Model:  -1.7887761603
  Diff:   8.6272118338e-09


In [1]:
import pandas as pd
import numpy as np

def inspect_features(df: pd.DataFrame, keywords=None, name="Subset"):
    """
    DataFrame에서 특정 키워드가 포함된 피처들의 건강 상태(NaN, Inf, Stats)를 점검합니다.
    
    Args:
        df: 검증할 DataFrame
        keywords: 컬럼 이름에 포함될 문자열 리스트 (예: ['O2C', 'O2H']). None이면 전체 검사.
        name: 출력할 때 표시할 그룹 이름
    """
    # 1. 검증할 컬럼 선별
    if keywords:
        target_cols = [c for c in df.columns if any(k in c for k in keywords)]
    else:
        target_cols = df.columns.tolist()
        
    # 중립화(_neut)된 피처는 통계적 특성이 다르므로 보통 따로 보는 게 좋습니다.
    # 여기서는 '_neut'가 '없는' 것만 골라내거나, 필요하면 로직을 수정하세요.
    # target_cols = [c for c in target_cols if "_neut" not in c] 

    print(f"\n{'='*10} [ 검증 그룹: {name} (총 {len(target_cols)}개 컬럼) ] {'='*10}")
    
    if not target_cols:
        print("❌ 해당 키워드를 가진 컬럼이 없습니다.")
        return

    subset = df[target_cols]
    
    # 2. 결측치(NaN) 체크
    nan_sum = subset.isna().sum()
    total_nans = nan_sum.sum()
    if total_nans > 0:
        print(f"⚠️ [NaN 경고] 총 {total_nans}개의 결측치가 발견되었습니다!")
        print(nan_sum[nan_sum > 0].sort_values(ascending=False).head(5)) # 상위 5개만 출력
    else:
        print("✅ 결측치(NaN): 없음")

    # 3. 무한대(Inf) 체크
    # pandas에서는 isinf가 바로 안 먹힐 수 있어 numpy 사용
    # select_dtypes로 수치형 데이터만 골라내서 확인
    numeric_cols = subset.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        inf_sum = np.isinf(subset[numeric_cols]).sum()
        total_infs = inf_sum.sum()
        if total_infs > 0:
            print(f"⚠️ [Inf 경고] 총 {total_infs}개의 무한대(Inf) 값이 발견되었습니다!")
            print(inf_sum[inf_sum > 0])
        else:
            print("✅ 무한대(Inf): 없음")
    
    # 4. 기초 통계량 (Min, Max, Mean, Std) 확인
    # 데이터가 너무 많으면 보기 힘드므로 Transpose(.T)해서 보여줌
    stats = subset.describe().T[['mean', 'min', 'max', 'std']]
    
    # 이상하게 튀는 값(Outlier)이 있는지 대략적으로 확인
    # 예: O2C(수익률)가 10000(100%)을 넘거나 -10000(-100%)보다 작은 경우 등
    print(f"\n📊 [통계 요약 - 상위 5개]")
    print(stats.head(5)) 
    
    # (선택) 특정 조건으로 '이상한 컬럼'만 뽑아보기
    # 예: 표준편차가 0인(값이 고정된) 컬럼 찾기
    constant_cols = stats[stats['std'] == 0].index.tolist()
    if constant_cols:
        print(f"\n⚠️ [주의] 값이 전혀 변하지 않는(Std=0) 컬럼이 있습니다: {constant_cols}")

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# 출력 옵션 설정
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

class FeatureInspector:
    def __init__(self, x_path, raw_dir, date_str, symbol):
        self.symbol = symbol
        self.date_str = date_str
        self.target_date = datetime.strptime(date_str, "%Y-%m-%d").date()
        
        # 1. Load Processed X Data
        print(f"[INFO] Loading X data: {x_path}")
        self.df_x = pd.read_hdf(x_path, key='x', mode='r')
        self.df_x = self.df_x[self.df_x['symbol'] == symbol].copy()
        
        # 2. Load Raw Data (Target Day + Previous Day)
        # 윈도우 계산을 위해 전날 데이터도 함께 로드해야 함
        prev_date_str = (self.target_date - timedelta(days=1)).strftime("%Y-%m-%d")
        raw_paths = [
            os.path.join(raw_dir, f"{prev_date_str}.h5"),
            os.path.join(raw_dir, f"{date_str}.h5")
        ]
        
        raw_list = []
        for p in raw_paths:
            if os.path.exists(p):
                print(f"[INFO] Loading Raw data: {p}")
                raw_list.append(pd.read_hdf(p, mode='r'))
            else:
                print(f"[WARN] Raw file missing: {p}")
                
        if not raw_list:
            raise ValueError("No raw data loaded.")
            
        self.df_raw = pd.concat(raw_list, ignore_index=True)
        self.df_raw = self.df_raw[self.df_raw['symbol'] == symbol].sort_values('start_time_ms').reset_index(drop=True)
        
        # 날짜 컬럼 추가 (보기 편하게)
        self.df_raw['datetime'] = pd.to_datetime(self.df_raw['start_time_ms'], unit='ms', utc=True)
        
    def inspect_window(self, target_ms, window_size):
        """
        특정 타임스탬프(target_ms) 기준으로 window_size만큼의 Raw Data를 잘라서 보여줌
        """
        # 1. Raw Data에서 해당 타임스탬프의 인덱스 찾기
        idx_list = self.df_raw.index[self.df_raw['start_time_ms'] == target_ms].tolist()
        
        if not idx_list:
            print(f"❌ Raw Data에 해당 타임스탬프({target_ms})가 없습니다.")
            return

        curr_idx = idx_list[0]
        start_idx = curr_idx - window_size + 1
        
        if start_idx < 0:
            print("⚠️ 윈도우가 데이터 시작점보다 앞서 있습니다. (잘림)")
            start_idx = 0

        # 2. Window Slicing (과거 N분 ~ 현재)
        raw_window = self.df_raw.iloc[start_idx : curr_idx + 1].copy()
        
        # 3. X Data에서 해당 타임스탬프 값 가져오기
        x_row = self.df_x[self.df_x['start_time_ms'] == target_ms]
        
        # --- 출력 ---
        target_dt = pd.to_datetime(target_ms, unit='ms', utc=True)
        print("\n" + "="*80)
        print(f"🔎 검증 시점: {target_dt} (Timestamp: {target_ms})")
        print(f"📦 Window Size: {window_size}분")
        print("="*80)

        print(f"\n1️⃣ [Raw Data Window] ({len(raw_window)} rows)")
        # 주요 컬럼만 출력
        disp_cols = ['datetime', 'open', 'high', 'low', 'close', 'quote_volume', 'taker_buy_quote']
        # 있으면 추가할 컬럼
        for c in ['mt_oi', 'pm_close']:
            if c in self.df_raw.columns: disp_cols.append(c)
            
        display(raw_window[disp_cols])
        
        # 검산 (O2C 직접 계산)
        o_first = raw_window['open'].iloc[0]
        c_last = raw_window['close'].iloc[-1]
        calc_o2c = (c_last / o_first - 1) * 10000
        
        print(f"\n2️⃣ [Calculated vs Feature Value]")
        print(f"   - (검산) 직접 계산한 {window_size}m_O2C: {calc_o2c:.4f} bp")
        
        feat_col = f"x_{window_size}m_O2C"
        if feat_col in x_row.columns:
            val_x = x_row[feat_col].values[0]
            print(f"   - (파일) 저장된 피처 {feat_col} : {val_x:.4f} bp")
            
            diff = abs(calc_o2c - val_x)
            if diff < 0.0001:
                print("   ✅ 일치합니다!")
            else:
                print(f"   ❌ 불일치! (차이: {diff:.4f}) - 중립화(Neut) 이전 값인지 확인하세요.")
        else:
            print(f"   ⚠️ X 파일에 {feat_col} 컬럼이 없습니다.")

        print("\n3️⃣ [Feature Row (All Columns)]")
        # 해당 윈도우와 관련된 피처만 필터링해서 보여주기
        rel_cols = [c for c in x_row.columns if f"_{window_size}m_" in c or c in ['start_time_ms', 'symbol']]
        display(x_row[rel_cols])


# =========================================================
# 사용 예시 (본인의 경로와 날짜로 수정하세요)
# =========================================================

# 설정
DATE = '2025-03-02'
SYMBOL = 'BTCUSDT'
TARGET_MS = 1740873600000  # 확인하고 싶은 시간 (ms)
WINDOW = 15                # 확인하고 싶은 윈도우 크기 (분)

# 경로 설정
data_dir = "C:\\Users\\박찬영\\Desktop\\수업\\딥러닝의 기초\\25_fall_deeplearning\\data"          # 데이터 루트 폴더
x_file = f"{data_dir}\\x\\{DATE}_x_top30.h5"
raw_dir = f"H:\\.shortcut-targets-by-id\\1f3z1EB8khva5IBuMRq-7U07pb-UCqvHI\\data\\1m_raw_data"

# 실행
try:
    inspector = FeatureInspector(x_file, raw_dir, DATE, SYMBOL)
    inspector.inspect_window(TARGET_MS, WINDOW)
except Exception as e:
    print(f"Error: {e}")

[INFO] Loading X data: C:\Users\박찬영\Desktop\수업\딥러닝의 기초\25_fall_deeplearning\data\x\2025-03-02_x_top30.h5
[INFO] Loading Raw data: H:\.shortcut-targets-by-id\1f3z1EB8khva5IBuMRq-7U07pb-UCqvHI\data\1m_raw_data\2025-03-01.h5
[INFO] Loading Raw data: H:\.shortcut-targets-by-id\1f3z1EB8khva5IBuMRq-7U07pb-UCqvHI\data\1m_raw_data\2025-03-02.h5
❌ Raw Data에 해당 타임스탬프(1741132620000)가 없습니다.


In [11]:
# 1. 데이터 로드
df = pd.read_hdf('C:\\Users\\박찬영\\Desktop\\수업\\딥러닝의 기초\\25_fall_deeplearning\\25_fall_deeplearning\\data\\x\\2025-03-01_x_top30.h5') # 본인 파일 경로

# 2. 그룹별로 검사 실행
# (1) Price 관련 피처 (O2C, H2C 등)
inspect_features(df, keywords=['H2L_Vol'], name="Price Features")

# (2) Volume 관련 피처
inspect_features(df, keywords=['AvgTrade', 'C2VWAP', 'NetTaker'], name="Volume/Flow Features")

# (3) OI 및 기타 지표
inspect_features(df, keywords=['OI_Chg', 'WhaleGap', 'Premium'], name="Sentiment Features")

# (4) 중립화(_neut)된 피처들만 따로 보기
inspect_features(df, keywords=['_neut'], name="Neutralized Features")


========== [ 검증 그룹: Price Features (총 20개 컬럼) ] ==========
✅ 결측치(NaN): 없음
✅ 무한대(Inf): 없음

📊 [통계 요약 - 상위 5개]
                     mean        min          max         std
x_1m_H2L_Vol    23.305975   0.011771   478.880259   18.535509
x_5m_H2L_Vol    55.216916   2.691926   619.752099   39.774647
x_15m_H2L_Vol   97.486891   8.412693   895.960280   67.777869
x_30m_H2L_Vol  139.070809   9.927646  1208.162634   96.043344
x_60m_H2L_Vol  198.586470  20.936277  1298.593879  134.954245

========== [ 검증 그룹: Volume/Flow Features (총 60개 컬럼) ] ==========
✅ 결측치(NaN): 없음
✅ 무한대(Inf): 없음

📊 [통계 요약 - 상위 5개]
                     mean         min           max         std
x_1m_AvgTrade  442.955105   19.770025  10365.602131  751.524336
x_1m_NetTaker    0.005399   -1.000000      1.000000    0.334187
x_1m_C2VWAP      0.008101 -123.294858     68.346288    6.155979
x_5m_AvgTrade  460.359232   29.192521   7775.042362  746.264966
x_5m_NetTaker   -0.001174   -0.729978      0.792172    0.207311

========== [ 검증 그룹: